In [1]:
import os, openpyxl
import pyperclip as pc
import pandas as pd

### 1. Data import

In [2]:
path = './SourceData/OpenCell/science.abi6983_table_s6.xlsx'

In [3]:
wb = openpyxl.load_workbook(path)
wb.sheetnames

['read_me',
 'localization_annotations',
 'localization_annotations_flatte',
 'Fig S7B']

In [4]:
# ws = wb['localization_annotations_flatte']
ws = wb['localization_annotations']
df = pd.DataFrame(ws.values)
df.head()

,0,1,2,3,4
0,target_name,ensg_id,annotations_grade_3,annotations_grade_2,annotations_grade_1
1,AAMP,ENSG00000127837,cytoplasmic,None,nucleoplasm
2,AATF,ENSG00000275700,nucleolus_gc,None,None
3,ABCE1,ENSG00000164163,cytoplasmic,vesicles,nucleoplasm
4,ABR,ENSG00000159842,cytoplasmic,vesicles,None


In [6]:
columns = df.iloc[0,[0,1,2,3,4]]
print(columns)

0            target_name
1                ensg_id
2    annotations_grade_3
3    annotations_grade_2
4    annotations_grade_1
Name: 0, dtype: object


In [7]:
df.columns = columns
df = df.iloc[1:, :]

In [8]:
# df.head(n=12)
df.tail()

,target_name,ensg_id,annotations_grade_3,annotations_grade_2,annotations_grade_1
1306,ZCCHC7,ENSG00000147905,nucleolus_gc,None,None
1307,ZCCHC9,ENSG00000131732,nucleolus_gc,None,None
1308,ZNF326,ENSG00000162664,nuclear_punctae;nucleolus_gc;nucleoplasm,None,None
1309,ZNF598,ENSG00000167962,cytoplasmic,None,er
1310,ZYX,ENSG00000159840,cell_contact,None,None


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 1 to 1310
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   target_name          1310 non-null   object
 1   ensg_id              1310 non-null   object
 2   annotations_grade_3  1306 non-null   object
 3   annotations_grade_2  470 non-null    object
 4   annotations_grade_1  339 non-null    object
dtypes: object(5)
memory usage: 51.3+ KB


#### How many nuclear membrane proteins in there?

In [55]:
df_fillna = df.fillna(value='NA')

In [62]:
df_fillna_NE = df_fillna[(df_fillna['annotations_grade_3'].str.contains('nuclear_membrane'))
                         |(df_fillna['annotations_grade_2'].str.contains('nuclear_membrane'))
                         |(df_fillna['annotations_grade_1'].str.contains('nuclear_membrane'))]

In [64]:
df_fillna_NE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 91 to 1293
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   target_name          38 non-null     object
 1   ensg_id              38 non-null     object
 2   annotations_grade_3  38 non-null     object
 3   annotations_grade_2  38 non-null     object
 4   annotations_grade_1  38 non-null     object
dtypes: object(5)
memory usage: 1.8+ KB


#### Realized that some of the proteins on OpenCell portal that is assgined to Nuclear membrane group actually is not localized to NE
#### E.g.CSE1L

### 2. ID conversion from Ensembl to UniprotKB

In [9]:
ensg_id_list = df['ensg_id'].unique()
len(ensg_id_list)

1310

In [10]:
ensg_id_list_spaced = ' '.join(ensg_id_list)

In [11]:
pc.copy(ensg_id_list_spaced)

#### Paste to ID mapping on Uniprot, filter for Reviewed, then excel output downloaded

#### All of 1310 genes were converted to 1312 IDs

In [12]:
# import of the output excel
df_id_converted = pd.read_excel('./SourceData/OpenCell/Intermediate/Ensg_id_to_uniprot_id.xlsx')
df_id_converted.info()

/Users/ShokenLEE/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312 entries, 0 to 1311
Data columns (total 7 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   yourlist:M202203266320BA52A5CE8FCD097CB85A53697A35478814R  1312 non-null   object
 1   isomap:M202203266320BA52A5CE8FCD097CB85A53697A35478814R    685 non-null    object
 2   Entry                                                      1312 non-null   object
 3   Entry name                                                 1312 non-null   object
 4   Protein names                                              1312 non-null   object
 5   Gene names                                                 1312 non-null   object
 6   Organism                                                   1312 non-null   object
dtypes: object(7)
memory usage: 71.9+ KB


In [13]:
df_id_converted = df_id_converted.drop(['isomap:M202203266320BA52A5CE8FCD097CB85A53697A35478814R'], axis=1)
df_id_converted = df_id_converted.rename(columns={'yourlist:M202203266320BA52A5CE8FCD097CB85A53697A35478814R': 'ensg_id'})
df_id_converted.head()

,ensg_id,Entry,Entry name,Protein names,Gene names,Organism
0,ENSG00000127837,Q13685,AAMP_HUMAN,Angio-associated migratory cell protein,AAMP,Homo sapiens (Human)
1,ENSG00000275700,Q9NY61,AATF_HUMAN,Protein AATF (Apoptosis-antagonizing transcrip...,AATF CHE1 DED HSPC277,Homo sapiens (Human)
2,ENSG00000164163,P61221,ABCE1_HUMAN,ATP-binding cassette sub-family E member 1 (2'...,ABCE1 RLI RNASEL1 RNASELI RNS4I OK/SW-cl.40,Homo sapiens (Human)
3,ENSG00000159842,Q12979,ABR_HUMAN,Active breakpoint cluster region-related protein,ABR,Homo sapiens (Human)
4,ENSG00000114331,Q15057,ACAP2_HUMAN,"Arf-GAP with coiled-coil, ANK repeat and PH do...",ACAP2 CENTB2 KIAA0041,Homo sapiens (Human)


#### What are duplicated ids?

In [20]:
df_id_converted_duplicate = df_id_converted[df_id_converted.duplicated(subset=['ensg_id'])]

In [21]:
df_id_converted_duplicate

,ensg_id,Entry,Entry name,Protein names,Gene names,Organism
220,ENSG00000147889,Q8N726,ARF_HUMAN,Tumor suppressor ARF (Alternative reading fram...,CDKN2A CDKN2 MLM,Homo sapiens (Human)
787,ENSG00000186184,P0DPB6,RPAC2_HUMAN,DNA-directed RNA polymerases I and III subunit...,POLR1D,Homo sapiens (Human)
900,ENSG00000152061,Q5R372,RBG1L_HUMAN,Rab GTPase-activating protein 1-like,RABGAP1L HHL KIAA0471,Homo sapiens (Human)
1020,ENSG00000127922,Q6ZVN7,SEML_HUMAN,"Putative protein SEM1, isoform 2",SEM1 C7orf76,Homo sapiens (Human)
1195,ENSG00000120802,P42167,LAP2B_HUMAN,"Lamina-associated polypeptide 2, isoforms beta...",TMPO LAP2,Homo sapiens (Human)


#### I searched those duplicated ensg_ids and realized that ID mapping yields two Entrys that correspond to a single gene name

#### Thus those duplicated ids can be removed

#### Generate a df without duplicate in ensg_id and will use this afterwords

In [22]:
df_id_converted_unique = df_id_converted.drop_duplicates(subset=['ensg_id'])

In [23]:
df_id_converted_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1307 entries, 0 to 1311
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ensg_id        1307 non-null   object
 1   Entry          1307 non-null   object
 2   Entry name     1307 non-null   object
 3   Protein names  1307 non-null   object
 4   Gene names     1307 non-null   object
 5   Organism       1307 non-null   object
dtypes: object(6)
memory usage: 71.5+ KB


### 3. Merge of df with df_id_converted

In [24]:
df_merged = df.merge(df_id_converted_unique, how='outer', left_on='ensg_id', right_on='ensg_id')

In [25]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1310 entries, 0 to 1309
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   target_name          1310 non-null   object
 1   ensg_id              1310 non-null   object
 2   annotations_grade_3  1306 non-null   object
 3   annotations_grade_2  470 non-null    object
 4   annotations_grade_1  339 non-null    object
 5   Entry                1307 non-null   object
 6   Entry name           1307 non-null   object
 7   Protein names        1307 non-null   object
 8   Gene names           1307 non-null   object
 9   Organism             1307 non-null   object
dtypes: object(10)
memory usage: 112.6+ KB


In [26]:
df_merged.head(n=10)

,target_name,ensg_id,annotations_grade_3,annotations_grade_2,annotations_grade_1,Entry,Entry name,Protein names,Gene names,Organism
0,AAMP,ENSG00000127837,cytoplasmic,None,nucleoplasm,Q13685,AAMP_HUMAN,Angio-associated migratory cell protein,AAMP,Homo sapiens (Human)
1,AATF,ENSG00000275700,nucleolus_gc,None,None,Q9NY61,AATF_HUMAN,Protein AATF (Apoptosis-antagonizing transcrip...,AATF CHE1 DED HSPC277,Homo sapiens (Human)
2,ABCE1,ENSG00000164163,cytoplasmic,vesicles,nucleoplasm,P61221,ABCE1_HUMAN,ATP-binding cassette sub-family E member 1 (2'...,ABCE1 RLI RNASEL1 RNASELI RNS4I OK/SW-cl.40,Homo sapiens (Human)
3,ABR,ENSG00000159842,cytoplasmic,vesicles,None,Q12979,ABR_HUMAN,Active breakpoint cluster region-related protein,ABR,Homo sapiens (Human)
4,ACAP2,ENSG00000114331,cytoplasmic,None,None,Q15057,ACAP2_HUMAN,"Arf-GAP with coiled-coil, ANK repeat and PH do...",ACAP2 CENTB2 KIAA0041,Homo sapiens (Human)
5,ACAP3,ENSG00000131584,cytoplasmic,mitochondria;big_aggregates,None,Q96P50,ACAP3_HUMAN,"Arf-GAP with coiled-coil, ANK repeat and PH do...",ACAP3 CENTB5 KIAA1716,Homo sapiens (Human)
6,ACLY,ENSG00000131473,cytoplasmic,nucleoplasm,None,P53396,ACLY_HUMAN,ATP-citrate synthase (EC 2.3.3.8) (ATP-citrate...,ACLY,Homo sapiens (Human)
7,ACTB,ENSG00000075624,membrane;cytoskeleton,cytoplasmic,None,P60709,ACTB_HUMAN,"Actin, cytoplasmic 1 (Beta-actin) [Cleaved int...",ACTB,Homo sapiens (Human)
8,ACTG1,ENSG00000184009,membrane;cytoskeleton,None,cytoplasmic,P63261,ACTG_HUMAN,"Actin, cytoplasmic 2 (Gamma-actin) [Cleaved in...",ACTG1 ACTG,Homo sapiens (Human)
9,ACTN1,ENSG00000072110,membrane;cytoskeleton,None,None,P12814,ACTN1_HUMAN,Alpha-actinin-1 (Alpha-actinin cytoskeletal is...,ACTN1,Homo sapiens (Human)


In [41]:
df_merged_clean = df_merged.dropna(subset=['Entry'])

In [42]:
df_merged_clean = df_merged_clean.drop(['target_name', 'Entry name'], axis=1)

In [43]:
df_merged_clean = df_merged_clean[['Entry', 'ensg_id', 'Protein names', 'Gene names',
       'Organism', 'annotations_grade_3', 'annotations_grade_2',
       'annotations_grade_1']]

In [45]:
df_merged_clean = df_merged_clean.fillna(value='NA')

In [77]:
df_merged_clean = df_merged_clean.rename(columns={'annotations_grade_3': 'OpenCell_top_grade', 'annotations_grade_2': 'OpenCell_mid_grade',
                       'annotations_grade_1': 'OpenCell_low_grade'})

In [86]:
df_merged_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1307 entries, 0 to 1309
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Entry               1307 non-null   object
 1   ensg_id             1307 non-null   object
 2   Protein names       1307 non-null   object
 3   Gene names          1307 non-null   object
 4   Organism            1307 non-null   object
 5   OpenCell_top_grade  1307 non-null   object
 6   OpenCell_mid_grade  1307 non-null   object
 7   OpenCell_low_grade  1307 non-null   object
dtypes: object(8)
memory usage: 91.9+ KB


#### Make sure how many nuclear membrane proteins in there?

In [81]:
df_merged_clean_NE = df_merged_clean[(df_merged_clean['OpenCell_top_grade'].str.contains('nuclear_membrane'))
                         |(df_merged_clean['OpenCell_mid_grade'].str.contains('nuclear_membrane'))
                         |(df_merged_clean['OpenCell_low_grade'].str.contains('nuclear_membrane'))]

In [82]:
df_merged_clean_NE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 90 to 1292
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Entry               38 non-null     object
 1   ensg_id             38 non-null     object
 2   Protein names       38 non-null     object
 3   Gene names          38 non-null     object
 4   Organism            38 non-null     object
 5   OpenCell_top_grade  38 non-null     object
 6   OpenCell_mid_grade  38 non-null     object
 7   OpenCell_low_grade  38 non-null     object
dtypes: object(8)
memory usage: 2.7+ KB


In [83]:
df_merged_clean_NE.head()

,Entry,ensg_id,Protein names,Gene names,Organism,OpenCell_top_grade,OpenCell_mid_grade,OpenCell_low_grade
90,Q8N6S5,ENSG00000177917,ADP-ribosylation factor-like protein 6-interac...,ARL6IP6 PFAAP1,Homo sapiens (Human),nuclear_membrane,big_aggregates,NA
131,O75531,ENSG00000175334,Barrier-to-autointegration factor (Breakpoint ...,BANF1 BAF BCRG1,Homo sapiens (Human),nuclear_membrane,nucleoplasm;chromatin,NA
257,Q8IYA6,ENSG00000169607,Cytoskeleton-associated protein 2-like (Radial...,CKAP2L,Homo sapiens (Human),centrosome;cytoskeleton,nucleoplasm;cytoplasmic,nuclear_membrane
397,Q9NPA8,ENSG00000120533,Transcription and mRNA export factor ENY2 (Enh...,ENY2 DC6,Homo sapiens (Human),nucleoplasm;nuclear_punctae;nuclear_membrane,NA,NA
552,P52294,ENSG00000114030,Importin subunit alpha-5 (Karyopherin subunit ...,KPNA1 RCH2,Homo sapiens (Human),nuclear_membrane;big_aggregates,nucleoplasm;cytoplasmic,NA


### 4. Export

In [84]:
df_merged_clean.to_csv('./Output/OpenCell_all_Hs.csv')

#### End of Note